In [ ]:
import folium
from folium import plugins
import requests
import json
import numpy as np
from pyArango.connection import *
from scipy.spatial.distance import pdist
import os
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('bmh')

In [ ]:
colors = '#58355e #e03616 #fff689 #cfffb0 #5998c5 #B5FFE1 #93E5AB #65B891 #4E878C #00241B'.split()
colors_for_gradient = '#1BE7FF #6EEB83 #E4FF1A #E8AA14 #FF5714'.split()
color_gradient = dict(zip(np.around(np.linspace(1,0,len(colors_for_gradient)),2),colors_for_gradient))

In [ ]:
conn = Connection(username='galvbnk', password='')
db   = conn['capstone']
col  = db['radar']

In [ ]:
def radar(coords, term, db_col, radius=20000, key=''):
    params = {'location': '{0},{1}'.format(*coords),
              'radius': radius,
              'keyword': term,
              'key': key}
    root = 'https://maps.googleapis.com/maps/api/place/radarsearch/json'
    
    r = requests.get(root, params=params)
    if r.status_code == requests.codes.ok:
        collection = {'type': 'FeatureCollection', 'features': [] }

        for v in r.json()['results']:
            collection['features'].append({'properties': {'name':term}, 'type': 'Feature', 'geometry': { 'type': 'Point',
                           'coordinates': [float(v['geometry']['location']['lng']),float(v['geometry']['location']['lat'])]}})
        doc = db_col.createDocument(initValues=params)
        doc['response'] = r.json()
        doc.save()
        return collection

In [ ]:
location = [39.7047, -105.0814]
res = {}
res['grocery'] = radar(location, 'king soopers', col, radius=25000)
res['chipotle'] = radar(location, 'chipotle', col, radius=25000)
res['college'] = radar(location, 'university', col, radius=25000)
res['park'] = radar(location, 'park', col, radius=25000)
res['oldchicago'] = radar(location, 'old chicago', col, radius=25000)
res['hospital'] = radar(location, 'hospital', col, radius=25000)

In [ ]:
for k,v in res.iteritems():
    print("{0}: {1}".format(k, len(v['features'])))

In [ ]:

res['hospital'] = radar(location, 'hospital', radius=25000)

In [ ]:
wm = folium.Map(location=location, zoom_start=13)
for idx, kv in enumerate(res.itervalues()):
    for feat in kv['features']:
        folium.CircleMarker(location=feat['geometry']['coordinates'][::-1], 
                            radius=2000,
                            fill_opacity=.075,
                            fill_color='#B49FCC',
                            color=None).add_to(wm)
        folium.CircleMarker(location=feat['geometry']['coordinates'][::-1], 
                            radius=200,
                            fill_opacity=1.0,
                            color= colors[idx],
                            fill_color=None).add_to(wm)
wm

In [ ]:
res['chipotle']['features'][0]['geometry']

In [ ]:
wm = folium.Map(location=location, zoom_start=13)
dists = {}
for k, v in res.iteritems():
    if k not in ['chipotle']: continue
    ps = [feat['geometry']['coordinates'][::-1] for feat in v['features']]
    dists[k] = pdist(ps, 'euclidean')
    wm.add_children(plugins.HeatMap(data=ps,
                                   blur=30,
                                   radius=90,
                                   gradient=color_gradient,
                                   min_opacity=.47,
                                   max_val=.6,
                                   max_zoom=20))
wm

In [ ]:
plt.figure(figsize=(13,15))
c = 1
for k,v in dists.iteritems():
    plt.subplot(4,2,c)
    plt.hist(v, bins=20, label=k);
    plt.axvline(v.mean(), c='r', linestyle='-.', label='{0:.2}'.format(v.mean()))
    c += 1
    plt.legend()